# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 20 2022 9:34AM,253471,18,INNOVA-8926192,Innova Softgel LLC,Released
1,Dec 20 2022 9:16AM,253470,10,CTF0011555,"Citieffe, Inc.",Released
2,Dec 20 2022 9:16AM,253469,19,MSP216911,"Methapharm, Inc.",Released
3,Dec 20 2022 9:16AM,253469,19,MSP216914,"Methapharm, Inc.",Released
4,Dec 20 2022 9:16AM,253469,19,MSP216915,"Methapharm, Inc.",Released
5,Dec 20 2022 9:05AM,253468,12,HH8553,Hush Hush,Released
6,Dec 20 2022 9:05AM,253467,12,HH8558,Hush Hush,Released
7,Dec 20 2022 9:04AM,253466,12,HH8545,Hush Hush,Released
8,Dec 20 2022 9:04AM,253465,12,HH8543,Hush Hush,Executing
9,Dec 20 2022 8:59AM,253460,10,0086257411,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,253467,Released,1
37,253468,Released,1
38,253469,Released,3
39,253470,Released,1
40,253471,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
253467,NaN,1.0
253468,NaN,1.0
253469,NaN,3.0
253470,NaN,1.0
253471,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
253377,0.0,1.0
253382,0.0,1.0
253383,0.0,1.0
253387,3.0,0.0
253391,0.0,10.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
253377,0,1
253382,0,1
253383,0,1
253387,3,0
253391,0,10


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,253377,0,1
1,253382,0,1
2,253383,0,1
3,253387,3,0
4,253391,0,10


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,253377,,1
1,253382,,1
2,253383,,1
3,253387,3,
4,253391,,10


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 20 2022 9:34AM,253471,18,Innova Softgel LLC
1,Dec 20 2022 9:16AM,253470,10,"Citieffe, Inc."
2,Dec 20 2022 9:16AM,253469,19,"Methapharm, Inc."
5,Dec 20 2022 9:05AM,253468,12,Hush Hush
6,Dec 20 2022 9:05AM,253467,12,Hush Hush
7,Dec 20 2022 9:04AM,253466,12,Hush Hush
8,Dec 20 2022 9:04AM,253465,12,Hush Hush
9,Dec 20 2022 8:59AM,253460,10,ISDIN Corporation
47,Dec 20 2022 8:50AM,253462,10,ISDIN Corporation
53,Dec 20 2022 8:50AM,253463,10,"Snap Medical Industries, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 20 2022 9:34AM,253471,18,Innova Softgel LLC,,1
1,Dec 20 2022 9:16AM,253470,10,"Citieffe, Inc.",,1
2,Dec 20 2022 9:16AM,253469,19,"Methapharm, Inc.",,3
3,Dec 20 2022 9:05AM,253468,12,Hush Hush,,1
4,Dec 20 2022 9:05AM,253467,12,Hush Hush,,1
5,Dec 20 2022 9:04AM,253466,12,Hush Hush,,1
6,Dec 20 2022 9:04AM,253465,12,Hush Hush,1,
7,Dec 20 2022 8:59AM,253460,10,ISDIN Corporation,,38
8,Dec 20 2022 8:50AM,253462,10,ISDIN Corporation,,6
9,Dec 20 2022 8:50AM,253463,10,"Snap Medical Industries, LLC",,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 20 2022 9:34AM,253471,18,Innova Softgel LLC,1,
1,Dec 20 2022 9:16AM,253470,10,"Citieffe, Inc.",1,
2,Dec 20 2022 9:16AM,253469,19,"Methapharm, Inc.",3,
3,Dec 20 2022 9:05AM,253468,12,Hush Hush,1,
4,Dec 20 2022 9:05AM,253467,12,Hush Hush,1,
5,Dec 20 2022 9:04AM,253466,12,Hush Hush,1,
6,Dec 20 2022 9:04AM,253465,12,Hush Hush,,1
7,Dec 20 2022 8:59AM,253460,10,ISDIN Corporation,38,
8,Dec 20 2022 8:50AM,253462,10,ISDIN Corporation,6,
9,Dec 20 2022 8:50AM,253463,10,"Snap Medical Industries, LLC",2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 20 2022 9:34AM,253471,18,Innova Softgel LLC,1,
1,Dec 20 2022 9:16AM,253470,10,"Citieffe, Inc.",1,
2,Dec 20 2022 9:16AM,253469,19,"Methapharm, Inc.",3,
3,Dec 20 2022 9:05AM,253468,12,Hush Hush,1,
4,Dec 20 2022 9:05AM,253467,12,Hush Hush,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 20 2022 9:34AM,253471,18,Innova Softgel LLC,1.0,NaN
1,Dec 20 2022 9:16AM,253470,10,"Citieffe, Inc.",1.0,NaN
2,Dec 20 2022 9:16AM,253469,19,"Methapharm, Inc.",3.0,NaN
3,Dec 20 2022 9:05AM,253468,12,Hush Hush,1.0,NaN
4,Dec 20 2022 9:05AM,253467,12,Hush Hush,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 20 2022 9:34AM,253471,18,Innova Softgel LLC,1.0,0.0
1,Dec 20 2022 9:16AM,253470,10,"Citieffe, Inc.",1.0,0.0
2,Dec 20 2022 9:16AM,253469,19,"Methapharm, Inc.",3.0,0.0
3,Dec 20 2022 9:05AM,253468,12,Hush Hush,1.0,0.0
4,Dec 20 2022 9:05AM,253467,12,Hush Hush,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4561853,112.0,3.0
12,1013866,3.0,1.0
15,1774010,56.0,0.0
16,253457,1.0,0.0
18,253471,1.0,0.0
19,1267197,58.0,0.0
21,1013590,4.0,0.0
22,253444,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4561853,112.0,3.0
1,12,1013866,3.0,1.0
2,15,1774010,56.0,0.0
3,16,253457,1.0,0.0
4,18,253471,1.0,0.0
5,19,1267197,58.0,0.0
6,21,1013590,4.0,0.0
7,22,253444,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,112.0,3.0
1,12,3.0,1.0
2,15,56.0,0.0
3,16,1.0,0.0
4,18,1.0,0.0
5,19,58.0,0.0
6,21,4.0,0.0
7,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,112.0
1,12,Released,3.0
2,15,Released,56.0
3,16,Released,1.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,21,22
Status,,,,,,,,
Executing,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
Released,112.0,3.0,56.0,1.0,1.0,58.0,4.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,21,22
0,Executing,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Released,112.0,3.0,56.0,1.0,1.0,58.0,4.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,21,22
0,Executing,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Released,112.0,3.0,56.0,1.0,1.0,58.0,4.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()